In [8]:
import pandas as pd
import numpy as np
import scanpy as sc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt

In [9]:
dataset = 'kumar-8-hard'
data_path = "./dataset/{}-filtered/10X/".format(dataset)
labels_path = "./dataset/{}-filtered/labels.csv".format(dataset)
markers_path = "./results/aggregate/{}/markers.csv".format(dataset)

In [10]:
adata = sc.read_10x_mtx(
    data_path,
    var_names='gene_symbols',
    cache=False
)

In [11]:
y_df = pd.read_csv(labels_path, index_col=0)
y_df

,cluster.ids
cell,
AAACCTGCACGACGAA,4
AAACCTGGTTCCACGG,5
AAACCTGTCTCGATGA,4
AAACGGGAGCGCTCCA,1
AAAGCAAAGTGAATTG,5
...,...
TTTGGTTCAGACTCGC,2
TTTGTCAAGTGGAGAA,2
TTTGTCACAAGCCATT,3


In [12]:
y_df = pd.DataFrame(adata.obs_names, columns=["cell"]).join(y_df, on="cell")
y_df

,cell,cluster.ids
0,AAACCTGCACGACGAA,4
1,AAACCTGGTTCCACGG,5
2,AAACCTGTCTCGATGA,4
3,AAACGGGAGCGCTCCA,1
4,AAAGCAAAGTGAATTG,5
...,...,...
612,TTTGGTTCAGACTCGC,2
613,TTTGTCAAGTGGAGAA,2
614,TTTGTCACAAGCCATT,3
615,TTTGTCAGTCGATTGT,4


In [14]:
mask = ~np.isnan(np.array(y_df['cluster.ids'])).reshape(-1)
mask[mask==False]

array([], dtype=bool)

In [15]:
y_df['cluster.ids'][mask]

0      4
1      5
2      4
3      1
4      5
      ..
612    2
613    2
614    3
615    4
616    5
Name: cluster.ids, Length: 617, dtype: int64

In [16]:
y = np.array(y_df['cluster.ids'][mask])

In [17]:
clusters, counts = np.unique(y, return_counts=True)
clusters, counts

(array([1, 2, 3, 4, 5]), array([ 98,  60,  62, 172, 225]))

In [18]:
weights = counts[np.argsort(clusters)]

In [24]:
def apply_classifier(X, y):
    clf = RandomForestClassifier()
    y_pred = cross_val_predict(clf, X, y, cv=5)
    f1 = f1_score(y, y_pred)
    return f1

markers_df = pd.read_csv(markers_path)
tools = markers_df.tool.unique()

f1_markers = {}
for tool in tools:
    f1_markers_tool = []
    for cluster in clusters:
        y_bin = np.array(y==cluster, dtype=int)
        markers = markers_df[
            (markers_df['cluster']==cluster) & (markers_df['tool']==tool)
           ].gene.unique()
        X_markers = adata[mask, markers].X.toarray()
        f1_markers_tool.append(apply_classifier(X_markers, y_bin))
    f1_markers[tool] = round((weights*np.array(f1_markers_tool)).sum()/weights.sum(), 3)

f1_all = []
for cluster in clusters:
    y_bin = np.array(y==cluster, dtype=int)
    X_all = adata[mask, ].X.toarray()
    f1_all.append(apply_classifier(X_all, y_bin))
f1_weighted = round((weights*np.array(f1_all)).sum()/weights.sum(), 3)

print("F1 weighted when training on markers")
print(f1_markers)
print("F1 weighted when training on all genes")
print(f1_weighted)

F1 weighted when training on markers
{'monocle': 0.973, 'scanpy': 0.974, 'seurat': 0.972, 'scvi': 0.974}
F1 weighted when training on all genes
0.982


In [39]:
# -------- train with increasing # of features taken from markers rank --------
scores = {}
step = 1
tools = ['monocle']
for tool in tools:
    print('Tool: ', tool)
    for cluster in clusters:
        print('Cluster: ', cluster)
        n_markers = len(markers_df[markers_df['cluster']==cluster])   
        y_bin = np.array(y==cluster, dtype=int)
        for i in range(step, n_markers+step, step):
            markers = markers_df[
            (markers_df['cluster']==cluster) & (markers_df['tool']==tool) & (markers_df['rank']<=i)
           ].gene.unique()
            X = adata[mask, markers].X.toarray()
            f1 = apply_classifier(X, y_bin)
            if tool not in scores:
                scores[tool] = {}
            if cluster not in scores[tool]:
                scores[tool][cluster] = []
            scores[tool][cluster].append(f1)
        # mean for each cluster
        scores[tool][cluster] = np.mean(scores[tool][cluster])


x_ticks = [i for i in range(step, n_markers+step, step)]
plt.figure(figsize=(10,5))
# plt.xticks(x_ticks, x_ticks)
# plt.plot(x_ticks, scores, marker='o')
# plt.ylabel("f1 weighted")
# plt.xlabel("# top features from each tool")
# plt.grid()
# plt.tight_layout()
# plt.show()
# plt.savefig(out_path+"score.eps")

# pd.DataFrame(scores, columns=['f1 weighted']).to_csv(out_path+"clf_score.csv")
scores

{'monocle': {1: 0.9006614950242668,
  2: 0.9900942125414391,
  3: 0.9689131504245367,
  4: 0.9875082653861628,
  5: 0.9839890813694623}}

In [36]:
len(scores['monocle'])

45

In [ ]:
# -------- train on all features and on markers --------

X_all = adata[mask, :].X.toarray()
report_all, feature_importance = apply_classifier(X_all, y)
report_markers, _ = apply_classifier(adata[mask, markers].X.toarray(), y)

pd.DataFrame(report_all).transpose().to_csv(out_path+"clf_report_all.csv")
pd.DataFrame(report_markers).transpose().to_csv(out_path+"clf_report_markers.csv")

sorted_idx = (-feature_importance).argsort()
rf_features_sorted = adata.var_names[sorted_idx]
importaces_sorted = feature_importance[sorted_idx]
pd.DataFrame(
    {'genes' : rf_features_sorted, 'importaces' : importaces_sorted}
    ).to_csv(out_path+"importances.csv")

In [ ]:
# -------- select n_markers*n_clusters features with RFE and RF --------

selector = RFE(RandomForestClassifier(), n_features_to_select=n_markers*n_clusters, step=0.5)
selector.fit(X_all, y)
sorted_idx = (selector.ranking_).argsort()
rfe_features_sorted = adata.var_names[sorted_idx]
pd.DataFrame(
    {'genes' : rfe_features_sorted}
    ).to_csv(out_path+"rfe_ranking.csv")

# automatically choose the number of features
# rfe = RFECV(estimator=RandomForestClassifier())
# rfe.fit(X_all, y)
# rfe.show()

# TODO:
# - valutare intersezione
# - valutare bontà del ranking allenando con markers più in basso nella classifica?

# important_features = features_sorted[0:120]
# important_features = [f for f, i in zip(features_sorted, importaces_sorted) if i >= importaces_sorted[119]]
# intersection = set(markers).intersection(set(important_features))

#        rank di randomforest                  rank del tool
# gene1         100 *                         * 1 - (0-20)
# gene2         1 *                            
# gene3         1 *                            
# gene4         0 *                            